# Como trabalhar com leis em Python

Recentemente estou envolvida em um projeto que prevê a análise de leis federais brasileiras usando python. Nesse primeiro momento, vou apresentar como podemos importar leis federais no python e utilizar as subdivisões da própria estrutura do texto para indexizar o conteúdo da lei.

## Por que dividir uma lei em pedaços menores?

Leis são documentos jurídicos e técnicos que normatizam e orientam diferentes aspectos da vida cotidiana. Por diversas razões estudiosos querem analisar e comparar o conteúdo de diferentes leis. 
<br><br>
Uma maneira de faze-lo é por meio da leitura minuciosa da lei em questão, mas qualquer pessoa que já tenha trabalhado direta ou indiretamente com legislações sabe que nem tudo. que está escrito é relevante em todos os casos. Digamos que você more em um condomínio e o síndico deseja proibir que moradores tenham animais de estimação nas unidades do condomínio. Para saber se o síndico pode fazer isso ou não, você deve consultar as leis 4.591/64 (Lei de Condomínios) e 10406/2002 (Novo Código Civil). Essas duas leis são gigantescas e versam sobre diversas matérias, mas tudo o que você quer saber está em provavelmente um ou dois parágrafos dentro de artigos específicos. 
<br><br>
Desta forma, muitas vezes não queremos estudar uma lei inteira, apenas uma parte da lei, de forma que, quando consideramos em transformar aquelas informaçnoes em dados, devemos saber a relevância ou o peso que cada conjunto de informações tem.
<br><br>

## Estrutura de um projeto de lei

[Este texto](https://www.politize.com.br/estrutura-das-leis-entenda/) do Guilhermo Glassman para o Politize-se explica de forma muito didática o que quer dizer cada parte da estrutura de uma lei. Resumo aqui de maneira bem rápida:

- **Entidade de origem:** indica se a lei é federal, estadual ou municipal
- **Referência da lei e data de criação**
- **Tipo de lei:** ordinária, complementar, medida provisória, emenda constitucional, decret legislativo e resolucão
- **Ementa:** Resumo do que é tratado na lei
- **Preâmbulo:** Justificativa ou contexto no qual a lei foi criada
- **Conteúdo:**
  * Título
  * Capítulo
  * Artigo:
    * Caput: Cabeça do artigo, texto que vem logo depois de "Artigo 1°", por exemplo, orienta o queas demais subdivisões vão falar
    * Parágrafos: Destacam aspectos importantes do artigo que não estão expressos no caput
    * Incíseos: Descrevem hipóteses em que a regra é aplicada
    * Alínea: Subdivisões dos incisos
  
Desta forma, sabemos que se procurarmos palavras-chave em leis, elas terão um peso e uma interpretaçnao de acordo com a parte da estrutura em que ela foi mencionada. 

## Leis e padrões

Aqui quero trabalhar com legislação federal. Felizmente, leis federais estão disponíveis integralmente em formato aberto sempre em sua última versão (considerando que essa lei tenha sido alterada por dispositivos futuros) e sua formatação não costuma conter erros, de maneira que podemos aproveitar a pontuação que demarca cada uma das camadas de um projeto de lei como expressões regulares para subdivisão em diferentes objetos.

## Mão na massa: Lei de Acesso à Informação

Vou trabalhar aqui com uma lei que conheço muito bem que é a Lei de Acesso à Informacão (LAI), que consultei inúmeras vezes quando trabalhava na [Transparência Brasil](www.transparencia.org.br) . A Lei 12.527/2011 está disponível [nesse link](http://www.planalto.gov.br/ccivil_03/_ato2011-2014/2011/lei/l12527.htm) . 

### 1. Importar o texto

Normalmente eu faria um scrap da página do governo com o ``requests.get`` , mas por alguma razão eu obtive diversas vezes um erro de conexão que impedia o scrap, e mesmo instalando pacotes de segurança o erro persistia. Como eu consigo realizar nessa máquina o scrap de outras páginas, o problema parece ser do lado do site do palácio do planalto. (Se alguém encontrar um fix para esse erro, entre em contato comigo) <br><br>
Como no meu projeto eu irei trabalhar com algumas poucas leis, eu posso me dar ao luxo de imprimir a página como pdf manualmente e importar no Python com o ``pdfminer``, que irá retornar uma grande string contendo toda a lei. 

In [1]:
import pdfminer
from pdfminer.high_level import extract_text

In [2]:
# Transformando a minha lei em uma STRING

lai = extract_text('lai.pdf')

# Evitando espaços duplos, para não haver problema nos splits mais pra frente:
import re

lai = re.sub(r'  ', ' ', lai)

print(repr(lai))

'L12527\n\n15/01/2021 16:03\n\nPresidência da República\nCasa Civil\nSubchefia para Assuntos Jurídicos\n\nLEI Nº 12.527, DE 18 DE NOVEMBRO DE 2011.\n\nRegula o acesso a informações previsto no inciso XXXIII\ndo art. 5º , no inciso II do § 3º do art. 37 e no § 2º do art.\n216 da Constituição Federal; altera a Lei nº 8.112, de 11\nde dezembro de 1990; revoga a Lei nº 11.111, de 5 de\nmaio de 2005, e dispositivos da Lei nº 8.159, de 8 de\njaneiro de 1991; e dá outras providências.\n\nMensagem de veto\n\nVigência\n\nRegulamento\n\nLei:\n\nA PRESIDENTA DA REPÚBLICA Faço saber que o Congresso Nacional decreta e eu sanciono a seguinte\n\nCAPÍTULO I\n\nDISPOSIÇÕES GERAIS\n\nArt. 1º Esta Lei dispõe sobre os procedimentos a serem observados pela União, Estados, Distrito Federal e\nMunicípios, com o fim de garantir o acesso a informações previsto no inciso XXXIII do art. 5º , no inciso II do § 3º do\nart. 37 e no § 2º do art. 216 da Constituição Federal.\n\nParágrafo único. Subordinam-se ao regim

In [3]:
# Verificando o tipo de string:

type(lai)

str

Vamos primeiro brincar um pouco, verificando alguns aspectos do texto da lei.

In [4]:
#ocorrência da palavra Transparência:

print("transparência" in lai)

True


In [5]:
# Ver se a palavra dinossauro não está presente na LAI

print("dinossauro" not in lai)

True


### Outras subdivisões

Como vocês podem ver comparando com a lei original, o arquivo importado contém todos os caracteres e informações contidas no PDF, com a excessão das figuras (por exemplo, brasão). Temos os seguintes símbolos:

- A primeire expressão **"Lei:"** determina o início do conteúdo
- A última expressão **"Brasília,"** (início do local e data) demarcao fim do conteúdo.
- **Art.** para artigo/caput
- **§ ou 'Parágrafo único.'** para parágrafo
- **(I -;II -;III -; IV - ...)** para incísos
- **(a -,b -,c - ...)** para alíneas


A partir desses síibolos devemos onstruir expressões regulares de forma orientar a subdivisão do conteúdo. devemos ter em mente as seguintes hipóteses de erro:

1. Pode haver um typo e o espaçamento esperado entre um símbolo e um travessão não ocorrem (ex: *I-* ao invés de *I -*)
2. No decorrer do texto da lei, outros artigos, parágrafos e leis são citadas, e não queremos que o texto divida ai. (Vamos precisar que ele considere o fato de ter pulado uma linha).

### Criando uma lista com todos os artigos

Para fins de anáise, digamos que eu queria criar uma lista em que cada elemento da lista seja um artigo, contendo seus respectivos parágrafos, incísos e alíneas.

In [6]:
lai_art = re.split(r'\n\nArt. ', lai)

lai_art[1]

'1º Esta Lei dispõe sobre os procedimentos a serem observados pela União, Estados, Distrito Federal e\nMunicípios, com o fim de garantir o acesso a informações previsto no inciso XXXIII do art. 5º , no inciso II do § 3º do\nart. 37 e no § 2º do art. 216 da Constituição Federal.\n\nParágrafo único. Subordinam-se ao regime desta Lei:\n\nI - os órgãos públicos integrantes da administração direta dos Poderes Executivo, Legislativo, incluindo as\n\nCortes de Contas, e Judiciário e do Ministério Público;\n\nII - as autarquias, as fundações públicas, as empresas públicas, as sociedades de economia mista e demais\n\nentidades controladas direta ou indiretamente pela União, Estados, Distrito Federal e Municípios.'

O elemento 0 são o preâmbulo, responsável, etc. os nossos artigos começam a partir do elemento 1. Como a nossa regex está precedida de dois pulares de linha (\n\n) garantimos que nenhuma menção à palavra Artigo no texto seja confundida com o início de um artigo.
A lei tem 47 artigos (+ preâmbulo = 48), vamos validar:

In [7]:
print(len(lai_art), type(lai_art))

48 <class 'list'>


#### Organizando um dataframe com caputs, incisos e alíneas

Digamos que para fins analíticos eu deseje separar o que é caput do que é inciso ou alínea, então eu terei um dataframe com o seguinte formado:

| conteudo | tipo | artigo_referencia | paragrafo_referencia
| ----------- | ----------- | ----------- | ----------- |
| Aplicam-se as disposições desta Lei, no que couber (...) | caput | 1 | NaN |
| A publicidade a que estão submetidas as entidades (...) | parágrafo | 16 | 1 |

In [8]:
# Pegar dentro de cada artigo,
# criar variável para citar artigo
# criar variável para enumerar tipo
# ir limpando de dentro dos artigos cada uma das coisas

import pandas as pd

In [9]:
type(lai_art)

list

In [10]:
###### preciso limpar as variáveis do lado de dentro e as tabelas dos caputs está feita

caput = pd.DataFrame({'conteudo': [], 
                   'tipo': [], 
                   'artigo_referencia': [] })

for i in lai_art:
    i = re.sub(r"\n\nParágrafo único.*", " ", i, flags = re.DOTALL) # tudo depois de
    i = re.sub(r"\n\n§  .*", " ", i, flags = re.DOTALL) # tudo depois de
    a = i[0:5]
    a = re.sub(r'[^0-9]+', '', a) 
    cap = pd.DataFrame({'conteudo': [i], 
                        'tipo': 'caput', 
                        'artigo_referencia': a })
    caput = pd.concat([caput , cap], axis = 0, sort=False)

caput = caput[1:]
caput

,conteudo,tipo,artigo_referencia
0,1º Esta Lei dispõe sobre os procedimentos a se...,caput,1
0,"2º Aplicam-se as disposições desta Lei, no que...",caput,2
0,3º Os procedimentos previstos nesta Lei destin...,caput,3
0,"4º Para os efeitos desta Lei, considera-se:\n\...",caput,4
0,5º É dever do Estado garantir o direito de ace...,caput,5
0,6º Cabe aos órgãos e entidades do poder públic...,caput,6
0,7º O acesso à informação de que trata esta Lei...,caput,7
0,8º É dever dos órgãos e entidades públicas pro...,caput,8
0,9º O acesso a informações públicas será assegu...,caput,9
0,10. Qualquer interessado poderá apresentar ped...,caput,10


Agora vamos fazer o inverso: retirar o texto dos caputs e manter apenas o texto dos parágrafos:

In [11]:
par = pd.DataFrame({'conteudo': [], 
                   'tipo': [], 
                   'artigo_referencia': [],
                   'paragrafo_referencia': []})

for n in range(len(lai_art)):
    i = lai_art[n]
    a = i[0:5]
    a = re.sub(r'[^0-9]+', '', a) 
    if "\n\nParágrafo único" in i:
        i = pd.DataFrame({'conteudo': re.split("\n\nParágrafo único", i)})
        i['paragrafo_referencia'] = 'paragrafo unico'
    else:
        i = pd.DataFrame({'conteudo': re.split("\n\n§", i)})
        i['paragrafo_referencia'] = i['conteudo'].astype(str).str[0:4]
        i['paragrafo_referencia'] = i['paragrafo_referencia'].str.replace(r'[^0-9]+', '')
    i['tipo'] = 'paragrafo'
    i['artigo_referencia'] = a
    i = i[1:]
    par = pd.concat([par , i], axis = 0, sort=False)

par

,conteudo,tipo,artigo_referencia,paragrafo_referencia
1,. Subordinam-se ao regime desta Lei:\n\nI - os...,paragrafo,1,paragrafo unico
1,. A publicidade a que estão submetidas as enti...,paragrafo,2,paragrafo unico
1,1º O acesso à informação previsto no caput nã...,paragrafo,7,1
2,2º Quando não for autorizado acesso integral ...,paragrafo,7,2
3,3º O direito de acesso aos documentos ou às i...,paragrafo,7,3
...,...,...,...,...
1,". Regulamento disporá sobre a composição, orga...",paragrafo,37,paragrafo unico
1,"1º A restrição de acesso a informações, em ra...",paragrafo,39,1
2,2º No âmbito da administração pública federal...,paragrafo,39,2
3,3º Enquanto não transcorrido o prazo de reava...,paragrafo,39,3


Agora vamos concatenar os dfs, tendo separados caputs e parágrafos:


In [12]:
import numpy as np

#juntando os dfs:
lai_df = pd.concat([par , caput], axis = 0, sort=False)

# retirando quebras de linha dupla, quebras de linha e whitespaces duplicados:
lai_df['conteudo'] = lai_df['conteudo'].str.replace(r'\n\n', ' ')
lai_df['conteudo'] = lai_df['conteudo'].str.replace(r'\n', ' ')

# Limpando início do conteúdo da lei
lai_df['conteudo'] = lai_df['conteudo'].str.replace(r'^[^a-zA-Z]+', ' ')

#pd.set_option('display.max_rows', None)
#lai_df.loc[lai_df['artigo_referencia'] == '1']
#print(repr(lai_df)) vendo se tinha qqooutro tipo de whitespace no df, mas nao tinha

# Ressetando o index
lai_df = lai_df.reset_index(drop=True, inplace=False)

lai_df

,conteudo,tipo,artigo_referencia,paragrafo_referencia
0,Subordinam-se ao regime desta Lei: I - os órg...,paragrafo,1,paragrafo unico
1,A publicidade a que estão submetidas as entid...,paragrafo,2,paragrafo unico
2,O acesso à informação previsto no caput não c...,paragrafo,7,1
3,Quando não for autorizado acesso integral à i...,paragrafo,7,2
4,O direito de acesso aos documentos ou às info...,paragrafo,7,3
...,...,...,...,...
113,"O inciso VI do art. 116 da Lei no 8.112, de 1...",caput,43,NaN
114,"O Capítulo IV do Título IV da Lei nº 8.112, d...",caput,44,NaN
115,"Cabe aos Estados, ao Distrito Federal e aos M...",caput,45,NaN
116,"Revogam-se: I - a Lei nº 11.111, de 5 de maio...",caput,46,NaN


In [43]:
# Validacão: verificando se todos os artigs estão aqui

y = pd.DataFrame(lai_df.groupby('artigo_referencia')['paragrafo_referencia'].count())
x = pd.read_csv("prova.csv")
x.set_index(['artigo_referencia'], inplace = True)

x.index = pd.to_numeric(x.index, errors='coerce')
y.index = pd.to_numeric(y.index, errors='coerce')
#y['artigo_referencia'] = str(y['artigo_referencia'])

#Renomeando
x = x.rename(columns={"paragrafo_referencia": "paragrafo_referencia_2"})

z = pd.merge(x, y, on='artigo_referencia')
#x.dtypes
z

#ótimo, mesmo resultado.

,paragrafo_referencia_2,paragrafo_referencia
artigo_referencia,,
1,1,1
10,3,3
11,6,6
12,1,1
13,1,1
14,0,0
15,1,1
16,3,3
17,2,2


Continuar:  Avisar quando parágrafo ou artigo é vetado.
    VETADO em maiúsculas
    problema na separação dos caputs: alguns parágrafos estão com n§ ao invés de nn§